# Functional Programming for Data Analysis

### Jim Pivarski

Second notebook: functional playground

C++ and Python are not functional languages.

Functional programming is a nebulously defined style, so there isn't a strict definition, but generally it involves working with expressions and not statements.

   * **Expression:** tree-like structure of nested function calls. Has a return value and can be used as an argument to a function. Examples: a FORTRAN formula, a diagrammed sentence, all of Lisp.
   * **Statement:** a command that either changes the computer's state or does nothing. Examples: Python's `for` and `if`, `move-robotic-arm`, all of assembly language.

This notebook will add methods to Python lists to make them easier to use for functional programming.

The goal will be to analyze data without ever writing a `for` loop or `if` statement.

In [ ]:
%matplotlib inline
import helpers.functional

In [ ]:
[1, 2, 3, 4, 5].map(lambda x: x**2)

To make it more real, let's work with real data (from the CMS public dataset).

In [ ]:
from helpers.functional import events

events.take(1)

Before trying to solve problems, we have to understand our toolset. Here are some of the methods that we've added to list:

In [ ]:
# not functional— a plain old function— but useful to peel off a few events to play with
events.take(2)

In [ ]:
# also not functional— but using a suffix rather than "len" makes it easier to read chains
events.take(12).size

In [ ]:
# aha! a real functional! but does it matter what order I put the "map" and the "take"?
events.map(lambda ev: ev.muons).take(5)

Filter is a very important functional in high energy physics.

In [ ]:
events.take(100).filter(lambda ev: ev.muons.size >= 2)

Flatten turns pesky lists-of-lists into simple lists.

In [ ]:
events.map(lambda ev: ev.muons).take(10)

"Flatmap" does "map" and "flatten" at the same time. It's more than a convenience— it has foundational importance (see [monadic bind](https://en.wikipedia.org/wiki/Monad_%28functional_programming%29)). For our purposes, we can think of it as a way of turning event ntuples into particle ntuples.

In [ ]:
events.flatmap(lambda ev: ev.muons).take(10)   # now a muon ntuple

"Reduce" is fundamentally different: it turns ntuples into aggregations (counts, sums, means, histograms...). All the other functionals we have seen so far turn ntuples into ntuples.

In [ ]:
events.map(lambda ev: ev.numPrimaryVertices).take(1000) \
      .reduce(lambda x, y: x + y) / 1000.0

In [ ]:
def weightAndPrimaryVertices(ev):
    return (1.0, ev.numPrimaryVertices)

def averageOnTheFly(x, y):
    wx, x = x
    wy, y = y
    return (wx + wy), (wx*x + wy*y)/(wx + wy)

Reduce has a hidden limitation: it combines pairs of elements, using the result as part of the next pair. So what if you want to make an aggregation that is of a different type than the elements you're combining?

In [ ]:
events.flatmap(lambda ev: ev.muons).take(2).reduce(lambda mu1, mu2: mu1.px + mu2.px)

In [ ]:
from histogrammar import *
from math import sqrt

hist = Bin(100, 0, 500, lambda muon: sqrt(muon.px**2 + muon.py**2 + muon.pz**2))

for muon in events.flatmap(lambda ev: ev.muons).take(10000):
    hist.fill(muon)     # not functional! this statement changes "hist"

hist.plot.matplotlib()

Well, there's an app for that.

In [ ]:
def newhist():
    return Bin(100, 0, 500, lambda muon: sqrt(muon.px**2 + muon.py**2 + muon.pz**2))

def filled(histogram, muon):
    h = newhist()
    h.fill(muon)
    return histogram + h

events.flatmap(lambda ev: ev.muons) \
      .take(100) \
      .aggregate(filled, zero=newhist())   # functional (but slow)

Generally, doing useful work with functional programming means being familiar with a toolset of handy combinators and knowing their mathematical properties.

Here's one: histogramming is important enough to be a standard combinator.

In [ ]:
events.flatmap(lambda ev: ev.muons).take(10000) \
      .Bin(100, 0, 500, lambda muon: sqrt(muon.px**2 + muon.py**2 + muon.pz**2)) \
      .plot.matplotlib()

Taking away the `for` loop is like taking away `GOTO`. The programmer has less power but the code better expresses the programmer's intent.

Solving the domain-specific problem (physics) and optimizing the calculation are better separated. If you _want_ to think about performance issues, you edit the underlying library, not the physics code.

Guided challenge: the following shows how to nest `for` loops with `map` combinators. However, the output has too much structure— eliminate it to plot the Z peak.

In [ ]:
def dimuon(muon1, muon2):                # trick to map and filter in one function:
    if muon1 == muon2: return []         # if the candidate doesn't pass cuts, return []
    else: return [(muon1 + muon2).mass]  # otherwise return singleton list

goodevents = events.filter(lambda ev: ev.muons.size > 1)

goodevents.map(lambda ev: ev.muons.map(lambda mu1: ev.muons.map(lambda mu2: dimuon(mu1, mu2)))).take(10)

Of course, for something as common as nested `for` loops, we'd have standard combinators.

In [ ]:
goodevents.flatmap(lambda ev: ev.muons.pairs(lambda mu1, mu2: (mu1 + mu2).mass)) \
          .take(1000).Bin(120, 60, 120, identity).plot.matplotlib()

In [ ]:
# help([].pairs)
# help([[], []].table)
help([[], []].zip)

Challenge: compute and plot `deltaR` for every muon-jet pair. (Don't use `for` loops!)

In [ ]:
from math import *

def deltaphi(particle1, particle2):
    return (particle1.phi - particle2.phi) % (2*pi) - pi

def deltaeta(particle1, particle2):
    return particle1.eta - particle2.eta

def deltaR(particle1, particle2):
    return sqrt(deltaphi(particle1, particle2)**2 + deltaeta(particle1, particle2)**2)

goodevents = events.filter(lambda ev: ev.muons.size > 0 and ev.jets.size > 0).take(1000)

In [ ]:
# goodevents. ...?

Functional programming changes how we think about data analysis; maybe hard to get used to at first, but in the long run it forces us to think about the necessary complexity of analysis problems rather than the accidental complexity.

   * **Necessary complexity:** How are my data structured? What mathematical operation do I want to apply? How can I restructure what I have to what I want?
   * **Accidental complexity:** What do I have to do to get the computer to compute it (efficiently)? How do I split up my tasks to run them in parallel?

Last example: generalizing histograms. In our examples, histograms are functionals that we combined with the likes of `map`, `filter`, and `reduce`. But making histogram elements combinational, we open them to generalization:

In [ ]:
muons = events.flatmap(lambda ev: ev.muons).take(1000)
muons.Count()

In [ ]:
muons.Deviate(lambda mu: mu.py)

In [ ]:
muons.Bin(5, -100, 100, lambda mu: mu.px, Count()).values

In [ ]:
muons.Bin(5, -100, 100, lambda mu: mu.px, Deviate(lambda mu: mu.py)).values

Challenge: below is what ROOT calls a "profile plot." How would you make a two-dimensional histogram (using what you know)?

In [ ]:
events.flatmap(lambda ev: ev.muons).take(10000) \
      .Bin(50, -100, 100, lambda mu: mu.px, Deviate(lambda mu: mu.py)) \
      .plot.matplotlib()

Next notebook: [nb3-spark.ipynb](nb3-spark.ipynb)